In [1]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.1/713.1 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.9/299.9 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 110.2 MB/s eta 0:

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.


In [2]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [3]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore(use_bm25=True)

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems in the [documentation page](https://docs.haystack.deepset.ai/docs/telemetry#how-can-i-opt-out). More information at [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry).
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [4]:
from haystack.utils import fetch_archive_from_http

doc_dir = "data/got"

fetch_archive_from_http(
    url="https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt1.zip",
    output_dir=doc_dir
)

INFO:haystack.utils.import_utils:Fetching from https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt1.zip to 'data/got'


True

In [5]:
import os
from haystack.pipelines.standard_pipelines import TextIndexingPipeline

files_to_index = [doc_dir + "/" + f for f in os.listdir(doc_dir)]
indexing_pipeline = TextIndexingPipeline(document_store)
indexing_pipeline.run_batch(file_paths=files_to_index)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
INFO:haystack.pipelines.base:It seems that an indexing Pipeline is run, so using the nodes' run method instead of run_batch.


Converting files:   0%|          | 0/183 [00:00<?, ?it/s]

Preprocessing:   0%|          | 0/183 [00:00<?, ?docs/s]

Updating BM25 representation...:   0%|          | 0/2356 [00:00<?, ? docs/s]

{'documents': [<Document: {'content': '\n\n\'\'\'\'\'The Princess and the Queen, or, the Blacks and the Greens\'\'\'\'\' is a novella by George R. R. Martin, published in the 2013 Tor Books anthology \'\'Dangerous Women\'\'. Set in the Westeros of Martin\'s \'\'A Song of Ice and Fire\'\' series, 200 years before the events of \'\'A Game of Thrones\'\' (1996), it chronicles the "continent-burning warfare" (called the "Dance of Dragons") that explodes between Targaryen Princess Rhaenyra and her stepmother, Queen Alicent.\n\nThe work is presented as the writing of the fictional Archmaester Gyldayn, also the "author" of Martin\'s 2014 novella \'\'The Rogue Prince\'\', a direct prequel to \'\'The Princess and the Queen\'\'.\n\n==Plot==\nWhen King Viserys I Targaryen dies, his widow Queen Alicent has their eldest son Prince Aegon crowned King Aegon II before Viserys\' daughter Rhaenyra, the only surviving child of his first marriage, can claim the Iron Throne herself. Though Rhaenyra is the 

In [6]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [7]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)


/usr/local/lib/python3.10/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.


INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [8]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [42]:
prediction = pipe.run(
    # query="Who is father of Arya Stark?",
    # query='Who created the Dothraki vocabulary?',
    # query='Who is the sister of Sansa?',
    query='Who is brother of Daenerys',
    # query='Who played Jon Snow',
    # query='Who is prime minister',
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

In [43]:
from pprint import pprint

pprint(prediction)

{'answers': [<Answer {'answer': 'Prince Rhaegar', 'type': 'extractive', 'score': 0.9446125626564026, 'context': "n the presumed-dead Aegon Targaryen, son of Daenerys's late brother Prince Rhaegar, intending to eventually install him as king of Westeros with Daene", 'offsets_in_document': [{'start': 554, 'end': 568}], 'offsets_in_context': [{'start': 68, 'end': 82}], 'document_ids': ['d7bfd7e98fb37aaf9456b66be109b847'], 'meta': {'_split_id': 3}}>,
             <Answer {'answer': 'Viserys', 'type': 'extractive', 'score': 0.9350496530532837, 'context': 'Rhaella died in childbirth, and Daenerys and her older brother Viserys were whisked away to Braavos by the Master of Arms of the Red Keep, Ser Willem ', 'offsets_in_document': [{'start': 63, 'end': 70}], 'offsets_in_context': [{'start': 63, 'end': 70}], 'document_ids': ['58256c00a2963b947cd472ed91d7a313'], 'meta': {'_split_id': 3}}>,
             <Answer {'answer': 'Viserys', 'type': 'extractive', 'score': 0.900044322013855, 'context': 'is

In [44]:
from haystack.utils import print_answers

print_answers(
    prediction,
    details="minimum" ## Choose from `minimum`, `medium`, and `all`
)

'Query: Who is brother of Daenerys'
'Answers:'
[   {   'answer': 'Prince Rhaegar',
        'context': "n the presumed-dead Aegon Targaryen, son of Daenerys's "
                   'late brother Prince Rhaegar, intending to eventually '
                   'install him as king of Westeros with Daene'},
    {   'answer': 'Viserys',
        'context': 'Rhaella died in childbirth, and Daenerys and her older '
                   'brother Viserys were whisked away to Braavos by the Master '
                   'of Arms of the Red Keep, Ser Willem '},
    {   'answer': 'Viserys',
        'context': 'is one of the last two surviving members (along with her '
                   'older brother, Viserys) of the House Targaryen, who, until '
                   'fourteen years before the events o'},
    {   'answer': 'Quentyn',
        'context': 'rs and the Iron Throne. To his daughter, Doran reveals '
                   'that her brother Quentyn has gone east to bring back "Fire '
                  